# Random_Forests1
Usando Decision_Trees1, logo só serve para variáveis dependentes do tipo 0/1

##### Vamos fazer 3 modelos 
- Faz apenas random das colunas a analisar por node (usando todas as amostras) (1)
- Faz apenas random das amostras para fazer uma tree (analisa tds as colunas por node) (2)
- Faz random das amostras e das colunas para cada Tree (3)

In [3]:
from Decision_Tree1 import *
import sklearn
df_=pd.read_csv('titanic_train.csv')

In [2]:
df_=df_.drop(['PassengerId','Name','SibSp','Parch','Ticket','Fare','Cabin','Embarked'],axis=1)

In [1]:
survived=df_['Survived']
df_=df_.drop(['Survived'],1)

NameError: name 'df_' is not defined

In [4]:
df_=pd.concat([df_,pd.DataFrame(survived)],sort=False,axis=1)

In [5]:
df_=df_.dropna().reset_index(drop=True)

In [6]:
df_1=df_.head(50)

## Random_Forest1.3

In [1]:
from Decision_Tree1 import *

class Forest:
    def __init__(self):
        self.trees=[]
        
    def add_tree(self,tree):
        (self.trees)+=[tree]
        
    def size(self):
        return len(self.trees)
    
    def get_tree(self,ind):
        return self.trees[ind]
    
    def merge_forests(self,forest):
        (self.trees)+=(forest.trees)
    
    
def random_df(df): #cria a tal Bootstrapped dataset 
    size=len(df.index)
    done=False
    new_df=pd.DataFrame()
    i=0
    while i<=(size+2) and not done: #size+2 é só pq sim
        ind=random.randrange(size)
        ds=pd.DataFrame(df.iloc[ind,:]).T
        new_df=pd.concat([new_df,ds],sort=False)
        if len(new_df.index)==size:
            done=True
    return new_df.reset_index(drop=True)
        

def build_forest(df,trees=1000,limit=None,forest=True,auto=True,columns_percentage=50,bootstrap=False,count=0):
    #limit e count servem para a função build_tree
    #bootstrap diz se cada vez que fazemos uma árvore fazemos random da dataframe ou não (T/F)
    F=Forest()
    for i in range(trees):
        if not(bootstrap):
            print('i',i)
            new_tree= build_tree(df,limit=limit,forest=forest,auto=auto,columns_percentage=columns_percentage,count=count)
            F.add_tree(new_tree)
        else:
            print('i',i)
            new_df=random_df(df)
            new_tree= build_tree(new_df,limit=limit,forest=forest,auto=auto,columns_percentage=columns_percentage,count=count)
            F.add_tree(new_tree)
    return F

def forest_classify(df,forest):
    #devolve lista de dicionários em que cada dict refer-se a uma amostra/linha
    #e os valores de cada dict referem-se as previsões de cada árvore em 
    #relação a essa amostra
    rows=len(df.index)
    bigout=[]
    for i in range(rows):
        output={0:0,1:0}
        row=pd.DataFrame(df.iloc[i,:]).T
        trees=forest.size()
        for n in range(trees):
            tree=forest.get_tree(n)
            val=test_tree(row,tree)[0]
            output[val]+=1
        bigout+=[output]
    return bigout

def forest_predictions(df, forest, var=1, limite=60):
    #devolve uma lista com as previsões da floresta
    #impondo um limite (percentagem) a uma variável
    #(só dizemos que se trata dessa variável se a percentagem de probabilidade)
    #for igual ou superior
    final=[]
    classi=forest_classify(df,forest)
    #print(classi)
    for i in classi:
        prob=round((i[var]/(forest.size()))*100)
        if prob>=limite:
            final+=[var]
        else:
            final+=[1-var]
    return final

def cmat_forest(df,forest,var=1,limite=60):
    real=list(df.iloc[:,-1])
    predicted=forest_predictions(df,forest,var,limite)
    print('predicted',predicted)
    cmat=np.array([[0,0],[0,0]])
    for i in range(len(real)):
        if real[i]==1 and predicted[i]==1:
            cmat[0][0]+=1
        elif real[i]==1 and predicted[i]==0:
            cmat[1][0]+=1
        elif real[i]==0 and predicted[i]==1:
            cmat[0][1]+=1
        elif real[i]==0 and predicted[i]==0:
            cmat[1][1]+=1
        else:
            print('Erro na cmat_forest')
            return 'Erro na cmat_forest'
    return cmat, pred_percent(cmat,1-var)

Este módulo usa pandas.Dataframe
E está feito para apenas prever variáveis categóricas 0/1
